In [20]:
import pandas as pd 
import tqdm as tqdm
import numpy as np 

Rewritten python version of the ELO_Calculation notebook by Flaty.

In [21]:
year = 2025
PATH  = "/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/DataPreparation/CompData/"
result_tourney_mens = pd.read_csv(PATH + f"march-machine-learning-mania-{year}/MNCAATourneyCompactResults.csv")
result_regular_mens = pd.read_csv(PATH  + f"march-machine-learning-mania-{year}/MRegularSeasonCompactResults.csv")
result_tourney_2nd_mens = pd.read_csv(PATH + f"march-machine-learning-mania-{year}/MSecondaryTourneyCompactResults.csv").drop(columns=['SecondaryTourney'])
#SecondaryTouney gives us the type of tournament which really isn't of much help
teams_mens = pd.read_csv(PATH + f"march-machine-learning-mania-{year}/MTeams.csv")

result_tourney_womens = pd.read_csv(PATH + f"march-machine-learning-mania-{year}/WNCAATourneyCompactResults.csv")
result_regular_womens = pd.read_csv(PATH + f"march-machine-learning-mania-{year}/WRegularSeasonCompactResults.csv")
teams_womens = pd.read_csv(PATH + f"march-machine-learning-mania-{year}/WTeams.csv")

In [22]:
result_tourney_mens['type'] = 'Tourney'
print("result_tourney_mens shape:", result_tourney_mens.shape)
print("DayNum range (Tourney):", result_tourney_mens['DayNum'].min(), "-", result_tourney_mens['DayNum'].max())
result_regular_mens['type'] = 'Regular'
print("result_regular_mens shape:", result_regular_mens.shape)
print("DayNum range (Regular):", result_regular_mens['DayNum'].min(), "-", result_regular_mens['DayNum'].max())
result_tourney_2nd_mens['type'] = 'SecondaryTourney'
print("result_tourney_2nd_mens shape:", result_tourney_2nd_mens.shape)
print("DayNum range (SecondaryTourney):", result_tourney_2nd_mens['DayNum'].min(), "-", result_tourney_2nd_mens['DayNum'].max())
result_merged_men = pd.concat([result_tourney_mens, result_regular_mens, result_tourney_2nd_mens])
print("After concatenation (result_merged_men) shape:", result_merged_men.shape)
result_merged_men = result_merged_men.sort_values(by=['Season', 'DayNum'])
print("After sorting (result_merged_men) shape:", result_merged_men.shape)
for col in ['WRating_before', 'LRating_before', 'WRating_after', 'LRating_after']:
    result_merged_men[col] = 50.0
print("After adding rating columns (result_merged_men) shape:", result_merged_men.shape)

result_tourney_mens shape: (2518, 9)
DayNum range (Tourney): 134 - 154
result_regular_mens shape: (192930, 9)
DayNum range (Regular): 0 - 132
result_tourney_2nd_mens shape: (1809, 9)
DayNum range (SecondaryTourney): 133 - 152
After concatenation (result_merged_men) shape: (197257, 9)
After sorting (result_merged_men) shape: (197257, 9)
After adding rating columns (result_merged_men) shape: (197257, 13)


In [23]:
result_regular_womens['type'] = 'Regular'
print("After adding type to result_regular_womens shape:", result_regular_womens.shape)
print("DayNum range (Regular):", result_regular_womens['DayNum'].min(), "-", result_regular_womens['DayNum'].max())
result_tourney_womens['type'] = 'Tourney'
print("After adding type to result_tourney_womens shape:", result_tourney_womens.shape)
print("DayNum range (Tourney):", result_tourney_womens['DayNum'].min(), "-", result_tourney_womens['DayNum'].max())
result_merged_women = pd.concat([result_regular_womens, result_tourney_womens])
print("After concatenation (result_merged_women) shape:", result_merged_women.shape)
result_merged_women = result_merged_women.sort_values(by=['Season', 'DayNum'])
print("After sorting (result_merged_women) shape:", result_merged_women.shape)
for col in ['WRating_before', 'LRating_before', 'WRating_after', 'LRating_after']:
    result_merged_women[col] = 50.0
print("After adding rating columns (result_merged_women) shape:", result_merged_women.shape)

After adding type to result_regular_womens shape: (137028, 9)
DayNum range (Regular): 0 - 132
After adding type to result_tourney_womens shape: (1650, 9)
DayNum range (Tourney): 135 - 155
After concatenation (result_merged_women) shape: (138678, 9)
After sorting (result_merged_women) shape: (138678, 9)
After adding rating columns (result_merged_women) shape: (138678, 13)


In [24]:
def update_elo_men(df, Mteams, K=1):
    teams = Mteams.copy()
    men_merged_df = df.copy()

    teams['Rating'] = 50.0
    teams['num'] = teams.index
    for i in tqdm.tqdm(range(len(men_merged_df))):#Iterates over every single game
        W_team_ID = int(men_merged_df.iloc[i,2])
        L_team_ID = int(men_merged_df.iloc[i,4])
        # print(W_team_ID,L_team_ID)
        WTeamRating_before = float(teams[teams['TeamID'] == W_team_ID]['Rating'].values[0])
        LTeamRating_before = float(teams[teams['TeamID'] == L_team_ID]['Rating'].values[0])
        WTeam_num = int(teams[teams['TeamID'] == W_team_ID]['num'].values[0])#The current rows of the winning and losing team 
        LTeam_num = int(teams[teams['TeamID'] == L_team_ID]['num'].values[0])
        WTeamRating_after = WTeamRating_before + K * (1 / (10**((WTeamRating_before - LTeamRating_before) / 10) + 1))
        LTeamRating_after = LTeamRating_before - K * (1 / (10**((WTeamRating_before - LTeamRating_before) / 10) + 1))
        #Find the team with the corresponding TeamID's rating
        men_merged_df.iloc[i, 9] = WTeamRating_before
        men_merged_df.iloc[i, 10] = LTeamRating_before
        men_merged_df.iloc[i, 11] = WTeamRating_after
        men_merged_df.iloc[i, 12] = LTeamRating_after

        teams.iloc[WTeam_num, 4] = WTeamRating_after #This column keeps tracking of the last ELO rating for each team
        teams.iloc[LTeam_num, 4] = LTeamRating_after
    return men_merged_df, teams





def update_elo_rating_women(df, Wteams, K=1):
    women_merged_df = df.copy()
    teams = Wteams.copy()
    teams['Rating'] = 50.0
    teams['num'] = teams.index
    for i in tqdm.tqdm(range(len(women_merged_df))):
        WTeamID = int(women_merged_df.iloc[i, 2])
        LTeamID = int(women_merged_df.iloc[i, 4])
        
        WTeamRating_before = float(teams[teams['TeamID'] == WTeamID]['Rating'].values[0])
        LTeamRating_before = float(teams[teams['TeamID'] == LTeamID]['Rating'].values[0])
        WTeam_num = int(teams[teams['TeamID'] == WTeamID]['num'].values[0])
        LTeam_num = int(teams[teams['TeamID'] == LTeamID]['num'].values[0])
        
        WTeamRating_after = WTeamRating_before + K * (1 / (10**((WTeamRating_before - LTeamRating_before) / 10) + 1))
        LTeamRating_after = LTeamRating_before - K * (1 / (10**((WTeamRating_before - LTeamRating_before) / 10) + 1))
        
        women_merged_df.iloc[i, 9] = WTeamRating_before
        women_merged_df.iloc[i, 10] = LTeamRating_before
        women_merged_df.iloc[i, 11] = WTeamRating_after
        women_merged_df.iloc[i, 12] = LTeamRating_after
        
        teams.iloc[WTeam_num, 2] = WTeamRating_after
        teams.iloc[LTeam_num, 2] = LTeamRating_after
    
    return women_merged_df, teams

In [ ]:
res_w, _ = update_elo_rating_women(result_merged_women, teams_womens)

100%|██████████| 100/100 [00:00<00:00, 1224.68it/s]


In [26]:
res_m, _ = update_elo_men(result_merged_men,teams_mens)

100%|██████████| 197257/197257 [03:42<00:00, 887.09it/s]


In [27]:
res_m.shape

(197257, 13)

In [28]:
res_m['type'].unique()

array(['Regular', 'Tourney', 'SecondaryTourney'], dtype=object)

In [29]:
res_m_regular = res_m[res_m["type"] == "Regular"]

tmp = res_m_regular[['Season', 'DayNum', 'WTeamID', 'WRating_before', 'WRating_after']].rename(
    columns={'WTeamID': 'TeamID', 'WRating_before': 'Rating_before', 'WRating_after': 'Rating_after'}
) #Separate winners

tmp2 = res_m_regular[['Season', 'DayNum', 'LTeamID', 'LRating_before', 'LRating_after']].rename(
    columns={'LTeamID': 'TeamID', 'LRating_before': 'Rating_before', 'LRating_after': 'Rating_after'}
)#and losers

#Combine winners and losers
tmp3 = pd.concat([tmp, tmp2], ignore_index=True)

#Find last game each team played, that is, their final rating before the tournament. So the maximum DayNum for each Season-TeamID combination
tmp4 = tmp3.groupby(['Season', 'TeamID'])['DayNum'].max().reset_index()

#Keep only last game for each team and filter from 2010 onward
elo_mens = tmp3.merge(tmp4, on=['Season', 'TeamID', 'DayNum'], how='inner')
elo_mens = elo_mens[elo_mens['Season'] >= 2010][['TeamID', 'Rating_after', 'Season']]
elo_mens = elo_mens.rename(columns={'Rating_after': 'Rating'})

#Merge with team names
elo_mens = elo_mens.merge(teams_mens, on='TeamID', how='left')#[["TeamID", "TeamName", "Season", "Rating"]]


In [30]:
elo_mens.shape

(5639, 6)

In [31]:


res_w_regular = res_w[res_w["type"] == "Regular"]

tmp = res_w_regular[["Season", "DayNum", "WTeamID", "WRating_before", "WRating_after"]].rename(
    columns={"WTeamID": "TeamID", "WRating_before": "Rating_before", "WRating_after": "Rating_after"}
)

tmp2 = res_w_regular[["Season", "DayNum", "LTeamID", "LRating_before", "LRating_after"]].rename(
    columns={"LTeamID": "TeamID", "LRating_before": "Rating_before", "LRating_after": "Rating_after"}
)

# Step 3: Combine winners and losers
tmp3 = pd.concat([tmp, tmp2], ignore_index=True)

# Step 4: Find last game each team played
tmp4 = tmp3.groupby(["Season", "TeamID"])["DayNum"].max().reset_index()

# Step 5: Keep only last game for each team and filter from 2010 onward
elo_womens = tmp3.merge(tmp4, on=["Season", "TeamID", "DayNum"], how="inner")
elo_womens = elo_womens[elo_womens["Season"] >= 2010][["TeamID", "Rating_after", "Season"]]
elo_womens = elo_womens.rename(columns={"Rating_after": "Rating"})

# Step 6: Merge with team names
elo_womens = elo_womens.merge(teams_womens, on="TeamID", how="left")#[["TeamID", "TeamName", "Season", "Rating"]]



In [32]:
Output_path = "/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/Feature_Eng/"
elo_mens.to_csv(Output_path + "mine_2025_EloRating_mens_10.csv", index=False)
elo_womens.to_csv(Output_path + "mine_2025_EloRating_womens_10.csv", index=False)